## LLM Agents that can Code with Skills (Tools)

*[Coding along with the Udemy online course AI Agents: Building Teams of LLM Agents that Work For You by Mohsen Hassan & Ilyass Tabiai]*

In [3]:
import pandas as pd

In [4]:
api_key = pd.read_csv("~/tmp/chat_gpt/autogen_agent_1.txt", sep=" ", header=None)[0][0]
print("Don't be a fool and send your api key to GitHub!")

Don't be a fool and send your api key to GitHub!


In [5]:
llm_config = {
    "model": "gpt-4o",
    "api_key": api_key
    }
print("Don't be a fool and send your api key to GitHub!")

Don't be a fool and send your api key to GitHub!


### Defining our skills

This time we'd like to give our agent(s) a predefined functionality they can use to carry out their task. This functionality is usually referred to by the terms `skills` or `tools`.

Skills (or tools) are python functions that must be defined with a **docstring**. A **docstring** is a rigorous way to comment and document functions. 

### Docstrings

Docstrings in Pythin are used to document Python modules, functions, classes and methods. 

"Documenting your Python code is all centered on docstrings. These are built-in strings that, when configured correctly, can help your users and yourself with your project’s documentation." ([realpython.com/documenting-python-code](https://realpython.com/documenting-python-code/#documenting-your-python-code-base-using-docstrings))

Example of how to use a docstring to document a function:
```python
def add_numbers(a, b):
    """
    This function adds two numbers and returns the result.

    Parameters:
    a (int): The first number to add.
    b (int): The second number to add.

    Returns:
    int: The sum of a and b.
    """
    return a + b
```

In this example the docstring is enclosed in triple quotes (double quotation marks, denoted by ") and provides 
- a brief description of the function,
- its parameters
- and its return value.

This information can later be accessed using the help() function attribute in Python, which can be very useful for understanding how to use the function properly. __Docstrings are how our agents will know how to use and what each function is for.__

### Skill no. 1: Get stock prices

For our next example we are going to define and give to our agents a skill (or function) that can retrieve stock prices using `yfinance`. Let's define with an adequate docstring:

In [1]:
# With user-defined functions
def get_stock_prices(stock_symbols, start_date, end_date):
    """Get the stock prices for the given stock symbols between
    the start and end dates.

    Args:
        stock_symbols (str or list): The stock symbols to get the
        prices for.
        start_date (str): The start date in the format 
        'YYYY-MM-DD'.
        end_date (str): The end date in the format 'YYYY-MM-DD'.
    
    Returns:
        pandas.DataFrame: The stock prices for the given stock
        symbols indexed by date, with one column per stock 
        symbol.
    """
    import yfinance

    stock_data = yfinance.download(
        stock_symbols, start=start_date, end=end_date
    )
    return stock_data.get("Close")

### Skill no. 2: Plot stock prices

Let's now define our second skill, a function that can plot prices of stocks that were retrieved using our previous skill.

In [2]:
def plot_stock_prices(stock_prices, filename):
    """Plot the stock prices for the given stock symbols.

    Args:
        stock_prices (pandas.DataFrame): The stock prices for the 
        given stock symbols.
    """
    import matplotlib.pyplot as plt

    plt.figure(figsize=(10, 5))
    for column in stock_prices.columns:
        plt.plot(
            stock_prices.index, stock_prices[column], label=column
                )
    plt.title("Stock Prices")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.grid(True)
    plt.legend()
    plt.savefig(filename)

### Command Line Executor

The __Command Line Executor__ is a tool from autogen that allows us to execute code locally on our laptop.

Since we will also execute code here, we are going to need a Command Line Executor again, __but this time, we are going to add an argument to register the two functions we want our agents to be able to use.__

In [6]:
from autogen.code_utils import create_virtual_env
from autogen.coding import CodeBlock, LocalCommandLineCodeExecutor

venv_dir = ".env_llm"
venv_context = create_virtual_env(venv_dir)

executor = LocalCommandLineCodeExecutor(
    virtual_env_context=venv_context,
    timeout=200,
    work_dir="coding",
    functions=[get_stock_prices, plot_stock_prices],
)
print(
    executor.execute_code_blocks(code_blocks=[CodeBlock(language="python", code="import sys; print(sys.executable)")])
)

exit_code=0 output='/Users/juergenkober/Projects/Python+Rust/_github-repos/working-with-financial-data/py-building-llm-agent-teams/Notebooks/.env_llm/bin/python\n' code_file='/Users/juergenkober/Projects/Python+Rust/_github-repos/working-with-financial-data/py-building-llm-agent-teams/Notebooks/coding/tmp_code_278b725fd664fea2ecbf552b6b17cf04.py'


## Agents definition

### Code Writer agent

We are going to define a code writer agent that is aware of the two new functions we defined. We need to __tell it about them in its system prompt__. 

To achieve this 
- we will load its default system prompt and add information to it about our functions
- and then save this new system prompt as its default system prompt.

In [7]:
from autogen import ConversableAgent, AssistantAgent

# Agent that writes code
code_writer_agent = AssistantAgent(
    name="code_writer_agent",
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

[autogen.oai.client: 10-06 14:22:03] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [9]:
# check the default system prompt message of the code writer
code_writer_agent_system_message = code_writer_agent.system_message
code_writer_agent_system_message

'You are a helpful AI assistant.\nSolve tasks using your coding and language skills.\nIn the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.\n    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.\n    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.\nSolve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.\nWhen using code, you must indicate the script type in the code block. The user cannot provide any oth

In [11]:
# the code executor we defined can automatically generate a prompt about the provided functions
# we have to let it know through its prompts of their existence
# the executor will automatically generate a prompt for all our functions as long as they're properly documented:
print(executor.format_functions_for_prompt())

You have access to the following user defined functions. They can be accessed from the module called `functions` by their function names.

For example, if there was a function called `foo` you could import it by writing `from functions import foo`

def get_stock_prices(stock_symbols, start_date, end_date):
    """Get the stock prices for the given stock symbols between
        the start and end dates.

        Args:
            stock_symbols (str or list): The stock symbols to get the
            prices for.
            start_date (str): The start date in the format 
            'YYYY-MM-DD'.
            end_date (str): The end date in the format 'YYYY-MM-DD'.

        Returns:
            pandas.DataFrame: The stock prices for the given stock
            symbols indexed by date, with one column per stock 
            symbol.
    """
    ...

def plot_stock_prices(stock_prices, filename):
    """Plot the stock prices for the given stock symbols.

        Args:
            stock_prices 

__We can now this prompt about our two functions to the default prompt and register it as the new default prompt of the code writer:__

In [12]:
# adding the functions prompt to the code writer agent's prompt
code_writer_agent_system_message += executor.format_functions_for_prompt()
# the complete prompt now contains additional information about our used defined functions
print(code_writer_agent_system_message)

You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feed

In [13]:
# updating the code writer agents's prompt:
code_writer_agent = ConversableAgent(
    name="code_writer_agent",
    system_message=code_writer_agent_system_message, # updating the code writer agents's prompt
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

[autogen.oai.client: 10-06 14:25:50] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


### Code Executor agent

We'll define the Code Executor agent the same way as last time.

In [14]:
# Code executor agent
code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    llm_config=False,
    code_execution_config={"executor": executor},
    human_input_mode="ALWAYS",
    default_auto_reply=
    "Please continue. If everything is done, reply 'TERMINATE'.",
)

### Coding Task

In [15]:
# finally we're going to define the coding task
import datetime

today = datetime.datetime.now().date()

task = f"Today is {today}."
"Create a plot showing the normalized price of NVDA and BTC-USD for the last 5 years "\
"with their 60 weeks moving average. "\
"Make sure the code is in markdown code block, save the figure"\
" to a file asset_analysis.png and who it. Provide all the code necessary in a single python bloc. "\
"Re-provide the code block that needs to be executed with each of your messages. "\
"If python packages are necessary to execute the code, provide a markdown "\
"sh block with only the command necessary to install them and no comments."

'Create a plot showing the normalized price of NVDA and BTC-USD for the last 5 years with their 60 weeks moving average. Make sure the code is in markdown code block, save the figure to a file asset_analysis.png and who it. Provide all the code necessary in a single python bloc. Re-provide the code block that needs to be executed with each of your messages. If python packages are necessary to execute the code, provide a markdown sh block with only the command necessary to install them and no comments.'

In [17]:
'''
# code that would execute our task but it's not supposed to run inside a notebook :)
chat_result = code_executor_agent.initiate_chat(
    code_writer_agent,
    message= task
)
'''

"\n# code that would execute our task but it's not supposed to run inside a notebook :)\nchat_result = code_executor_agent.initiate_chat(\n    code_writer_agent,\n    message= task\n)\n"